In [2]:
!pip install nbformat --quiet
!pip install --upgrade langchain --quiet
!pip install openai --quiet
!pip install python-dotenv --quiet

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from MC_LIME import *
from langchain.prompts import ChatPromptTemplate
import re
from chat_open_ai import chat_openai_model

# External functions
from functions_pipeline import *
from prompt_templates import TEMPLATES

### Metavariables

In [2]:
NUM_WEEKS=5

LIME_bool=False
MC_LIME_bool=False
CEM_bool=True
average_class=False
course="dsp_001"

## Import data & model
- Which student data to pick?
- Feature scores of the student
- LIME results OR MC-LIME results (depending on the strategy)

In [3]:
# CHANGE HERE THE FEATURE SET
feature_set=f"features/{course}_features_{str(NUM_WEEKS)}_weeks.csv"
features = pd.read_csv(feature_set)
features.shape

(17512, 140)

In [5]:
model_name="meta/meta-llama-3-70b-instruct"

In [6]:
print(f"MODEL NAME: {model_name}")
if "/" in model_name:
    model_name_file_path = model_name.split("/")[-1]

file_path = f"data/{model_name_file_path}_answers_{course}_completed.csv"

print(f"FILE PATH to save: {file_path}")

print(f"model name to save: {model_name_file_path}\n\n\n")

MODEL NAME: meta/meta-llama-3-70b-instruct
FILE PATH to save: data/meta-llama-3-70b-instruct_answers_dsp_001_completed.csv
model name to save: meta-llama-3-70b-instruct





In [7]:
# GET PREDICTION MODEL
model_path = 'models/'

loaded_model=get_model(model_path, course, NUM_WEEKS)

2024-08-07 13:13:02.808209: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-08-07 13:13:02.808267: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-07 13:13:02.808280: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-07 13:13:02.808356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-07 13:13:02.808406: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-08-07 13:13:03.680080: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outpu

(None, 140)


In [10]:
instances = get_feature_indices(course, features)

In [15]:
# Retrieve data for student idx
lime_path=f"uniform_eq_results_ori_{str(NUM_WEEKS)}_weeks/LIME/{course}/dataframes/"
cem_path=f"uniform_eq_results_ori_5_weeks/CEM/{course}/"
mapping={"Pass" : 0, "Fail" : 1}


In [11]:

# Fix this
instances = instances[instances != 44]

# Add 5192
instances=np.append(instances, 5192)

assert 44 not in instances
assert 5192 in instances

In [13]:
idx=5192

# Create list of prompts

In [16]:
template=TEMPLATES["template"]
goal_definition = TEMPLATES["goal_definition"]
features_description = get_features_description(features)
model_description = TEMPLATES["model_description"]
post_hoc_description = define_post_hoc_description(LIME_bool, MC_LIME_bool, CEM_bool)
format_instructions = TEMPLATES["format_instructions"]
course_description = TEMPLATES[f"course_description_{course}"]
data_string = get_data(course, loaded_model, idx, instances, features, lime_path, cem_path, MC_LIME_bool, LIME_bool, CEM_bool, average_class)
conversation_template = TEMPLATES["conversation_template"]

presentation_examples = TEMPLATES["presentation_examples"]
presentation_template = TEMPLATES["presentation_template"]

2024-08-07 13:15:31.355965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-07 13:15:31.792688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-07 13:15:31.852554: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-07 13:15:31.987898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step


2024-08-07 13:15:32.047289: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/davide/Desktop/EPFL/XAI Semester Project/llm-xai/functions_pipeline.py:187: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_orig=mapping[results["real value"][0]]


In [17]:
print(data_string)

MODEL PREDICTION: fail, with 99.745601% of confidence.

FEATURE IMPORTANCES
These are the features found important by CEM: 
regularity_peak_dayhour_InWeek5 - 0.941664
number_sessions_InWeek5 - 0.818363
regularity_peak_dayhour_InWeek4 - 0.764466
number_sessions_InWeek3 - 0.734278
number_sessions_InWeek4 - 0.727545
frequency_action_Video.Load_InWeek5 - 0.690784
time_sessions_sum_InWeek4 - 0.606090
regularity_peak_dayhour_InWeek3 - 0.449346
time_sessions_mean_InWeek2 - 0.419732
time_sessions_mean_InWeek1 - 0.382991

FEATURE VALUES
The relevant feature values found by CEM for the student are included below: {'regularity_peak_dayhour_InWeek5': 0.025333148585514, 'number_sessions_InWeek5': 0.1816365452729089, 'regularity_peak_dayhour_InWeek4': 0.1981650447492319, 'number_sessions_InWeek3': 0.0, 'number_sessions_InWeek4': 0.0908182726364544, 'frequency_action_Video.Load_InWeek5': 0.9990009990009991, 'time_sessions_sum_InWeek4': 0.023412474372623, 'regularity_peak_dayhour_InWeek3': 0.0, 'time_

In [18]:
instructions_dataset=pd.read_csv("prompting_strategies.csv")
instructions_dataset=instructions_dataset[['name', 'Communication Prompt', 'LLM-generated prompt']]
instructions_dataset = instructions_dataset[instructions_dataset['name'] != 'hesslow_contrastive']
instructions_dataset = instructions_dataset.rename(columns={'Communication Prompt': 'instruction'})
instructions_dataset = instructions_dataset.rename(columns={'LLM-generated prompt': 'presentation_instruction'})
instructions_dataset

,name,instruction,presentation_instruction
0,relevance_selection,Use the social science theory “Relevance-Based...,• Relevant Causes: Say to the student which ca...
1,abnormal_conditions,Use the social science theory “Abnormal Condit...,• Abnormal Causes: Tell the student which caus...
2,pearl_explanation,Use the social science theory “Pearl’s model o...,• External Factors: Identify the external fact...
3,necessity_robustness_selection,Use the social science theory “Necessity and R...,• Necessary Causes: Identify the most necessar...
4,contrastive_explanation,Use the social science theory “Contrastive Exp...,• Target Event: Explain the event that did occ...
6,base_contrastive_explanation,Use the social science theory “Contrastive Exp...,• Target Event (P): Describe the event that di...
7,statistical_relevance,Use the social science theory “Statistical Rel...,"• Likely Causes: List the factors (A, B, and C..."
8,chain_of_thought,Let’s think step-by-step,NaN


In [19]:
from prompt_class import prompt_class

prompt_object=prompt_class(template, instructions_dataset)

context_string=prompt_object.context_partial_prompt( 
    data_string,
    features_description, 
    goal_definition, 
    model_description, 
    post_hoc_description, 
    course_description,
    )

In [20]:
prompt_object.add_explanation_template(presentation_template)

ChatPromptTemplate(input_variables=['course_description', 'presentation_instruction'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['course_description', 'presentation_instruction'], template="\nGiven this report, I want you to write a shorter version using the theory of feedback from Hattie et al.:\n\nWhere Am I Going? - A brief description of the student's performance and explicitly state the learning goal\nHow Am I Doing? - A brief description of the explanation findings\nWhere to Next? - Two recommended actions that the student can take that are specific to weeks of the course (make connections between previous weeks and upcoming weeks)\n\nThe student who is going to interact with you is the same student that the data is about, so use the tone of a professor who is talking to a student in a comforting and personable manner. \n\nFollow the instructions underneath in the INSTRUCTIONS section.\n\nINSTRUCTIONS In the explanation findings section (How am I

In [21]:
prompt_object.presentation_context_partial_prompt(course_description)

ChatPromptTemplate(input_variables=['presentation_instruction'], partial_variables={'course_description': '\nThe course the student is taking is Digital Signal Processing 1, which is a Master’s level course over 10 weeks under the topic of Electrical Engineering and Computer Science.\nThis is the course content:\n\nWEEK 1\n\nSKILLS: Digital Signals\nTOPICS: Welcome to the DSP course, Introduction to signal processing\n\nWEEK 2\n\nSKILLS: Digital Signals\nTOPICS: Discrete time signals, The complex exponential, The Karplus-Strong Algorithm\n\nWEEK 3\n\nSKILLS: Hilbert (Linear Alg.)\nTOPICS: Motivation and Examples, From Euclid to Hilbert, Hilbert Space, properties and bases, Hilbert Space and approximation\n\nWEEK 4\n\nSKILLS: DFT, DTFT DFS, DTFT: intuition and properties, FFT\nTOPICS: Exploration via a change of basis, The Discrete Fourier Transform (DFT), DFT Examples, DFT, DFS, DTFT, DTFT formalism, Relationship between transforms, Sinusoidal modulation, FFT: history and algorithms\n\

In [22]:
prompt_object.create_prompts_list(instructions_dataset)

,name,instruction,prompt,presentation_prompt
0,relevance_selection,Use the social science theory “Relevance-Based...,"[content=""\nYou are an AI assistant that analy...","[content='\nGiven this report, I want you to w..."
1,abnormal_conditions,Use the social science theory “Abnormal Condit...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
2,pearl_explanation,Use the social science theory “Pearl’s model o...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
3,necessity_robustness_selection,Use the social science theory “Necessity and R...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
4,contrastive_explanation,Use the social science theory “Contrastive Exp...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
6,base_contrastive_explanation,Use the social science theory “Contrastive Exp...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
7,statistical_relevance,Use the social science theory “Statistical Rel...,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."
8,chain_of_thought,Let’s think step-by-step,"[content=""\nYou are an AI assistant that analy...","[content=""\nGiven this report, I want you to w..."


In [213]:
# Old instances list
#instances = [123, 230, 528, 784, 1908, 2404, 4584, 5192, 6353, 6478, 7504, 7967, 8286, 9502, 9749, 12204, 12521, 13359, 14030, 16920]

### Print prompts (for testing)

In [23]:
from IPython.display import display, Markdown, Latex
import warnings
warnings.filterwarnings("ignore")

def display_prompt(prompting_strategies, strategy, prompt_type):
    # Determine the column to display based on prompt_type
    if prompt_type == "prompt1":
        column_name = "prompt"
    elif prompt_type == "prompt2":
        column_name = "presentation_prompt"
    else:
        raise ValueError("Invalid prompt_type. Choose 'prompt1' or 'prompt2'.")

    # Filter the dataframe to get the specific strategy
    selected_prompt = prompting_strategies[prompting_strategies["name"] == strategy][column_name].values[0]
    selected_prompt = selected_prompt[0].content
    
    # Display the prompt using Markdown
    display(Markdown(selected_prompt))

Add here prompt1 or prompt2 to print the explanation selection or presentation prompts

In [24]:
strategy = 'relevance_selection'
display_prompt(prompt_object.prompting_strategies, strategy, "prompt2")


Given this report, I want you to write a shorter version using the theory of feedback from Hattie et al.:

Where Am I Going? - A brief description of the student's performance and explicitly state the learning goal
How Am I Doing? - A brief description of the explanation findings
Where to Next? - Two recommended actions that the student can take that are specific to weeks of the course (make connections between previous weeks and upcoming weeks)

The student who is going to interact with you is the same student that the data is about, so use the tone of a professor who is talking to a student in a comforting and personable manner. 

Follow the instructions underneath in the INSTRUCTIONS section.

INSTRUCTIONS In the explanation findings section (How am I going?) explicitly use the following structure:
• Relevant Causes: Say to the student which causes you selected as relevant based on the question, context, and your background. Be explicit.
• New Information: Say to the student which information you thought they knew already, such as "Assuming that you know…" and highlight aspects that haven't been previously communicated to the student. Be clear and honest.
Finally:
• Say to the student that you focused on the most relevant causes that provide new insights. Specify which causes you selected and why they are important.


The course the student is taking is Digital Signal Processing 1, which is a Master’s level course over 10 weeks under the topic of Electrical Engineering and Computer Science.
This is the course content:

WEEK 1

SKILLS: Digital Signals
TOPICS: Welcome to the DSP course, Introduction to signal processing

WEEK 2

SKILLS: Digital Signals
TOPICS: Discrete time signals, The complex exponential, The Karplus-Strong Algorithm

WEEK 3

SKILLS: Hilbert (Linear Alg.)
TOPICS: Motivation and Examples, From Euclid to Hilbert, Hilbert Space, properties and bases, Hilbert Space and approximation

WEEK 4

SKILLS: DFT, DTFT DFS, DTFT: intuition and properties, FFT
TOPICS: Exploration via a change of basis, The Discrete Fourier Transform (DFT), DFT Examples, DFT, DFS, DTFT, DTFT formalism, Relationship between transforms, Sinusoidal modulation, FFT: history and algorithms

WEEK 5

SKILLS: Ideal Filters, Filter Design
TOPICS: Linear Filters, Filtering by example, Filter stability, Frequency response, Ideal filters, Filter design - Part 1: Approximation of ideal filters, Realizable filters, Implementation of digital filters, Filter design - Part 2: Intuitive filters, Filter design - Part 3: Design from specs, Real-time processing, Dereverberation and echo cancelation

WEEK 6

SKILLS: Modulation, Interpolation & Sampling
TOPICS: Introduction to continuous-time paradigm, Interpolation, The space of bandlimited signals, Sampling and aliasing: Introduction, Sampling and aliasing, Discrete-time processing and continuous-time signals, Another example of sampled acquisition

WEEK 7

SKILLS: Multirate
TOPICS: Stochastic signal processing, Quantization, A/D and D/A conversion

WEEK 8

SKILLS: DFT, DTFT DFS, Ideal Filters
TOPICS: (Revisiting the topics of week 4 with additional context) Image processing, Image manipulations, Frequency analysis, Image filtering, Image compression, The JPEG compression algorithm

WEEK 9

SKILLS: Modulation, Quantization
TOPICS: Digital communication systems, Controlling the bandwidth, Controlling the power, Modulation and demodulation, Receiver design, ADSL

WEEK 10

SKILLS: Applications
TOPICS: The end, Startups and DSP, Acknowledgements, Conclusion video


5 weeks of the course have concluded.

Follow these rules:

do not include the output of the model or a prediction
if you include a feature name, describe what it means
try to be as concise as possible
do not include the headers from hattie et al. in the response, but keep the structure
limit yourself to 200 words
do not include a sign-off, simply include the main content

To communicate this intervention most effectively, use Grice's maxims of conversation.

do not say things that you believe to be false
do not say things for which you do not have sufficient evidence.
do not add information that is not relevant
Only say what is relevant
be orderly
Your goal is to explain to the student what they should do to improve their performance in the course in the best way possible. Follow the instructions above.



# Calls to the GPT API

In [17]:
!pip install -qU langchain-groq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.23 which is incompatible.


In [25]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [32]:
from chat_replicate import chat_replicate_model
# DEFINE LANGCHAIN MODEL
llm_class=chat_replicate_model(model_name, 0.5)
conversation=llm_class.define_chat_model(conversation_template)

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}


In [33]:
test_prompt=prompt_object.prompting_strategies["prompt"][2]
test_prompt

[HumanMessage(content="\nYou are an AI assistant that analyzes struggling students behavior to help them in their learning trajectories and facilitate learning in the best possible way. \nYou have many information to help you in your goal:\n- A model prediction of student performance at the end of the course, in the form of “pass” or “fail”.\n- A post-hoc explainable AI approach that identifies which features are important to this student’s prediction.\n- Data in the form of student's features over 5 weeks that were used by the model. You will always see the most relevant features selected by the explainer\n- The course content and structure.\n- Detailed instructions on how to reason.\n\nThe model you are using is a recurrent neural network that is trained on predicting the student performance at the end of the course, in the form of “pass” or “fail”. The features of that the model are using are derived from student behavior:\ndelay_lecture:The average delay in viewing video lectures a

In [34]:
conversation.memory.chat_memory.messages=[]

In [92]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    HumanMessage(content="You're a helpful assistant"),
    AIMessage(content="What is the purpose of model regularization?"),
]

conversation.memory.chat_memory.messages=messages

In [135]:
# Here we call the model with the main prompt
result1=conversation.predict(input=test_prompt)

'### Report on Student Performance Analysis\n\n#### 1. Identification of Exogenous and Endogenous Variables\n\n**Exogenous Variables:**\n- **delay_lecture:** This is an external factor as it depends on the release schedule of the lectures.\n- **ratio_clicks_weekend_day:** This ratio depends on the student\'s personal schedule and preference for studying on weekends versus weekdays.\n- **total_clicks:** The overall engagement of the student, which is influenced by personal motivation and interest.\n- **total_clicks_problem:** This is influenced by the number of problems available and the student\'s willingness to engage with them.\n- **total_clicks_video:** Similar to total_clicks_problem, this is influenced by the availability of video content and the student\'s engagement.\n- **student_speed:** This is influenced by the student\'s individual pace of learning and problem-solving.\n- **number_sessions:** This is influenced by the student\'s study habits and time management.\n\n**Endogen

In [181]:
conversation.memory.chat_memory.messages

[HumanMessage(content=[{'content': "\nYou are an AI assistant that analyzes struggling students behavior to help them in their learning trajectories and facilitate learning in the best possible way. \nYou have many information to help you in your goal:\n- A model prediction of student performance at the end of the course, in the form of “pass” or “fail”.\n- A post-hoc explainable AI approach that identifies which features are important to this student’s prediction.\n- Data in the form of student's features over 5 weeks that were used by the model. You will see alwyays the most relevant features selected by LIME.\n- The course Digital Signal Processing 1 content and structure.\n- Detailed instructions on how to reason.\n\nThe model you are using is a recurrent neural network that is trained on predicting the student performance at the end of the course (Week 10), in the form of “pass” or “fail”. The features of that the model are using are derived from student behavior:\ndelay_lecture:T

In [255]:
test_presentation=prompt_object.prompting_strategies["presentation_prompt"][2]
test_presentation

[HumanMessage(content="\nGiven this report, I want you to write a shorter version using the theory of feedback from Hattie et al.:\n\nWhere Am I Going? - A brief description of the student's performance and explicitly state the learning goal\nHow Am I Doing? - A brief description of the explanation findings\nWhere to Next? - Two recommended actions that the student can take that are specific to weeks of the course (make connections between previous weeks and upcoming weeks)\n\nThe student who is going to interact with you is the same student that the data is about, so use the tone of a professor who is talking to a student in a comforting and personable manner. \n\nFollow the instructions underneath in the INSTRUCTIONS section.\n\nINSTRUCTIONS In the explanation findings section (How am I going?) explicitly use the following structure:\n• External Factors: Identify the external factors (exogenous variables) that influence the result. Explain why these factors are important and how they

In [256]:
# Here we call the model with the presentation prompt
result2=conversation.predict(input=test_presentation)

In [29]:
with open(f'data/MC_LIME_results_{course}.json', 'r') as file:
    MC_LIME_results = json.load(file)

no_MCLIME = [int(idx) for idx, result in MC_LIME_results.items() if not result]
no_MCLIME


[1, 5296, 192, 44, 757, 264]

In [30]:
instances=[5192]

In [36]:
# GIVEN THE INDICES, WE ITERATE OVER THE STUDENTS AND THE EXPLANATION STRATEGIES TO GET THE RESPONSES
# MANY CALLS TO THE OPENAI API, BE CAREFUL WITH THE BILLING

import warnings
warnings.filterwarnings("ignore")
prompt_object.add_explanation_template(presentation_template)
#presentation_prompt=prompt_object.explanation_prompt_generator(presentation_instruction, presentation_question)

columns = ['student', 'explainer', 'strategy', 'first_prompt', 'response1', 'response2']
results_temp = pd.DataFrame(columns=columns)

explainer_list = {
    "CEM": {"MC_LIME_bool": False, "LIME_bool": False, "CEM_bool": True},
    "LIME": {"MC_LIME_bool": False, "LIME_bool": True, "CEM_bool": False},
    "MC-LIME": {"MC_LIME_bool": True, "LIME_bool": False, "CEM_bool": False}   
}


for idx in instances:
    for explainer in explainer_list:
        MC_LIME_bool = explainer_list[explainer]["MC_LIME_bool"]
        LIME_bool = explainer_list[explainer]["LIME_bool"]
        CEM_bool = explainer_list[explainer]["CEM_bool"]

        # Check if the student has a MCLIME explanation, if not, we only use LIME
        if idx in no_MCLIME and explainer == "MC-LIME":
            MC_LIME_bool = False
            LIME_bool = True
            CEM_bool = False
        
        data_string = get_data(course, loaded_model, idx, instances, features, lime_path, cem_path, MC_LIME_bool, LIME_bool, CEM_bool)
        post_hoc_description = define_post_hoc_description(LIME_bool, MC_LIME_bool, CEM_bool)
        context_string=prompt_object.context_partial_prompt( 
            data_string,
            features_description, 
            goal_definition, 
            model_description, 
            post_hoc_description, 
            course_description,
            )
        prompt_object.create_prompts_list(instructions_dataset)
        
        for index, row in prompt_object.prompting_strategies.iterrows():
                print(f"Student: {idx}, Explainer: {explainer}, Strategy: {row['name']}\n")
                main_prompt=row['prompt']
                presentation_prompt=row['presentation_prompt']
                # Call your function to interact with GPT-4
                llm_class=chat_replicate_model(model_name, 0.5)
                conversation=llm_class.define_chat_model(conversation_template)
                response1=conversation.predict(input=main_prompt)
                response2=conversation.predict(input=presentation_prompt)
                # Append the results to the DataFrame
                new_row={
                    'student': idx,
                    'explainer': explainer,
                    'strategy': row['name'],
                    'first_prompt': main_prompt[0].content,
                    'second_prompt': presentation_prompt[0].content,
                    'response1': response1,
                    'response2': response2,
                }

                results_temp = pd.concat([results_temp, pd.DataFrame([new_row])], ignore_index=True)


1/1 [==============================] - 0s 152ms/step
Student: 5192, Explainer: CEM, Strategy: relevance_selection

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}
Student: 5192, Explainer: CEM, Strategy: abnormal_conditions

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}
Student: 5192, Explainer: CEM, Strategy: pearl_explanation

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}
Student: 5192, Explainer: CEM, Strategy: necessity_robustness_selection

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}
Student: 5192, Explainer: CEM, Strategy: contrastive_explanation

Replicate
Params: {'model': 'meta/meta-llama-3-70b-instruct', 'model_kwargs': {'temperature': 0.5}}
Student: 5192, Explainer: CEM, Strategy: base_contrastive_explanation

Replicate
Params: {'model': 'meta/meta-llama-3-70b-in

13 seconds for a gpt3.5 call, 50 seconds for a gpt4 call

In [37]:
# results_temp = results_temp[results_temp['student'] != 1908]
results_temp

,student,explainer,strategy,first_prompt,response1,response2,second_prompt
0,5192,CEM,relevance_selection,\nYou are an AI assistant that analyzes strugg...,"\n\nBased on the model prediction, explainer, ...",\n\nHere's a shorter version of the report usi...,"\nGiven this report, I want you to write a sho..."
1,5192,CEM,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,"\n\nBased on the model prediction, explainer, ...",\n\nHere is a shorter version of the report us...,"\nGiven this report, I want you to write a sho..."
2,5192,CEM,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,\n\nBased on the provided data and instruction...,\n\nHere is a shorter version of the report us...,"\nGiven this report, I want you to write a sho..."
3,5192,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,\n\nReport: Identifying Key Factors Affecting ...,\n\nHere's a shorter version of the report usi...,"\nGiven this report, I want you to write a sho..."
4,5192,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,\n\nReport: Understanding the Struggling Stude...,\n\nHere's a shorter version of the report usi...,"\nGiven this report, I want you to write a sho..."
5,5192,CEM,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,\n\n**Contrastive Explanation Report**\n\n**Re...,\n\nHere is a shorter version of the report us...,"\nGiven this report, I want you to write a sho..."
6,5192,CEM,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,\n\nBased on the feature importances and value...,\n\nHere is a shorter version of the report us...,"\nGiven this report, I want you to write a sho..."
7,5192,CEM,chain_of_thought,\nYou are an AI assistant that analyzes strugg...,\n\nBased on the model prediction and the feat...,\n\nHere is a shorter version of the report us...,"\nGiven this report, I want you to write a sho..."
8,5192,LIME,relevance_selection,\nYou are an AI assistant that analyzes strugg...,\n\nBased on the model prediction and the expl...,\n\nHere's a shorter version of the report usi...,"\nGiven this report, I want you to write a sho..."
9,5192,LIME,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,\n\nReport:\n\nBased on the model prediction a...,\n\nHere's a shorter version of the report usi...,"\nGiven this report, I want you to write a sho..."


In [42]:
dsp_results=pd.read_csv("data/final_results/meta-llama-3-70b-instruct_answers_dsp_001_completed.csv")
dsp_results

,student,explainer,strategy,first_prompt,second_prompt,response1,response2
0,23,LIME,relevance_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Key Insights for the Student's Lea...,\n\nWhere Am I Going?\nYou're doing well in th...
1,23,LIME,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Insights for a Str...,\n\nHere's a shorter version of the report usi...
2,23,LIME,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Insights from Mode...,\n\nHere's a shorter version of the report usi...
3,23,LIME,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Insights into Student Behavior and...,\n\nHere's a shorter version of the report usi...
4,23,LIME,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\n**Contrastive Explanation Report**\n\n**Ta...,\n\nHere is a shorter version of the report us...
...,...,...,...,...,...,...,...
2371,1684,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Factors Affecting ...,\n\nWhere Am I Going?\nYou're currently strugg...
2372,1684,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Contrastive Explanation of Student...,\n\nHere's a shorter version of the report usi...
2373,1684,CEM,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Insights for a Str...,\n\nHere's a shorter version of the report usi...
2374,1684,CEM,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nBased on the feature importances and value...,\n\nHere's a shorter version of the report usi...


In [43]:
dsp_results=pd.concat([dsp_results, results_temp], ignore_index=True)
dsp_results

,student,explainer,strategy,first_prompt,second_prompt,response1,response2
0,23,LIME,relevance_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Key Insights for the Student's Lea...,\n\nWhere Am I Going?\nYou're doing well in th...
1,23,LIME,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Insights for a Str...,\n\nHere's a shorter version of the report usi...
2,23,LIME,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Insights from Mode...,\n\nHere's a shorter version of the report usi...
3,23,LIME,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Insights into Student Behavior and...,\n\nHere's a shorter version of the report usi...
4,23,LIME,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\n**Contrastive Explanation Report**\n\n**Ta...,\n\nHere is a shorter version of the report us...
...,...,...,...,...,...,...,...
2395,5192,MC-LIME,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Identifying Key Factors Influencin...,\n\nHere's a feedback report using the theory ...
2396,5192,MC-LIME,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\n**Contrastive Explanation Report**\n\n**Ta...,\n\nHere is a shorter version of the report us...
2397,5192,MC-LIME,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nReport: Contrastive Explanation of Student...,\n\nHere is a shorter version of the report us...
2398,5192,MC-LIME,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",\n\nBased on the feature importances and value...,\n\nHere is a shorter version of the report us...


In [45]:
dsp_results.loc[0]["response2"]

"\n\nWhere Am I Going?\nYou're doing well in the Digital Signal Processing 1 course, and your goal is to continue performing strongly and pass the course.\n\nHow Am I Doing?\nRelevant Causes: I've identified competency alignment, competency anticipation, and regular periodicity in Week 5 as key factors contributing to your success. \nNew Information: Assuming you know that you've been performing well in the course, I want to highlight that your strong competency alignment and anticipation in Week 5 are particularly noteworthy. \nI focused on these causes because they provide new insights into your strengths and areas for improvement.\n\nWhere to Next?\nTo build on your strengths, I recommend that you review the material from Week 4, specifically the Discrete Fourier Transform (DFT) and its applications, to reinforce your understanding of the topics that will be revisited in Week 8. Additionally, make sure to allocate dedicated time for practicing problems and quizzes in Week 6 to maint

In [44]:
dsp_results.to_csv("data/final_results/meta-llama-3-70b-instruct_answers_dsp_001_completed.csv", index=False)

In [129]:
file_path = f"data/gpt4o-answers_{course}.csv"
if os.path.exists(file_path):
    results_df = pd.read_csv(file_path)
else:
    columns = ['student', 'explainer', 'strategy', 'first_prompt', 'second_prompt', 'response1', 'response2']
    results_df = pd.DataFrame(columns=columns)

In [130]:
results_df

,student,explainer,strategy,first_prompt,second_prompt,response1,response2
0,23,CEM,relevance_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Student Performance Analysis Report\n\n**S...,### Feedback on Your Performance and Next Step...
1,23,CEM,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Prediction\n...,### Feedback on Your Performance\n\nYou've sho...
2,23,CEM,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Analysis\n\n...,### Feedback on Your Performance and Next Step...
3,23,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Analysis Report on Student Performance Pre...,"Your performance so far has been promising, es..."
4,23,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",**Report on Student Performance Analysis**\n\n...,You've been doing well in the Digital Signal P...
...,...,...,...,...,...,...,...
2371,1684,MC-LIME,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Analysis Report on Student's Predicted Fai...,### Feedback on Your Performance\n\nYou've bee...
2372,1684,MC-LIME,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Analysis\n\n...,You've been struggling in the Digital Signal P...
2373,1684,MC-LIME,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Analysis\n\n...,Your performance so far in the Digital Signal ...
2374,1684,MC-LIME,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Analysis Report on Student Performance Pre...,Your performance so far indicates that you're ...


In [131]:
# Append the rows of results_temp to results_df using pd.concat
results_df = pd.concat([results_df, results_temp], ignore_index=True)

# Display the DataFrame
results_df

,student,explainer,strategy,first_prompt,second_prompt,response1,response2
0,23,CEM,relevance_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Student Performance Analysis Report\n\n**S...,### Feedback on Your Performance and Next Step...
1,23,CEM,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Prediction\n...,### Feedback on Your Performance\n\nYou've sho...
2,23,CEM,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Analysis\n\n...,### Feedback on Your Performance and Next Step...
3,23,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Analysis Report on Student Performance Pre...,"Your performance so far has been promising, es..."
4,23,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",**Report on Student Performance Analysis**\n\n...,You've been doing well in the Digital Signal P...
...,...,...,...,...,...,...,...
2395,5192,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Prediction A...,### Feedback on Your Performance and Next Step...
2396,5192,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Report on Student Performance Analysis\n\n...,You've been doing well to keep up with the cou...
2397,5192,CEM,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",**Contrastive Explanation Report**\n\n**Questi...,Where Am I Going?\n\nYou've been working hard ...
2398,5192,CEM,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,"\nGiven this report, I want you to write a sho...",### Analysis Report on Student Performance Pre...,### Feedback on Your Performance and Next Step...


In [143]:
results_df["student"].value_counts()

student
1090    24
2560    24
2608    24
3170    24
1562    24
Name: count, dtype: int64

In [132]:
results_df.to_csv(f'data/gpt4o-answers_{course}.csv', index=False)

In [16]:
course="dsp_001"
results_df=pd.read_csv(f'data/gpt4o-answers_{course}.csv')
initial_instances=results_df["student"].unique()

results_df = results_df[results_df['student'].isin(instances)]
initial_instances

array([  123,   230,   528,   784,  2404,  4584,  1908,  5192,  6353,
        6478,  7504,  7967,  8286,  9502,  9749, 12204, 12521, 13359,
       14030, 16920])

In [18]:
results_df.to_csv(f'gpt4o-answers_{course}.csv', index=False)

In [225]:
results_df=pd.read_csv(f'gpt4o-answers_{course}.csv')
results_df

,student,explainer,strategy,first_prompt,response1,response2
0,1090,LIME,relevance_selection,\nYou are an AI assistant that analyzes strugg...,### Report on Student Performance Analysis\n\n...,NaN
1,1090,LIME,abnormal_conditions,\nYou are an AI assistant that analyzes strugg...,### Analysis Report on Student Performance Pre...,NaN
2,1090,LIME,pearl_explanation,\nYou are an AI assistant that analyzes strugg...,### Analysis Report on Student Performance Pre...,NaN
3,1090,LIME,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,### Analysis Report on Student's Predicted Fai...,NaN
4,1090,LIME,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,### Contrastive Explanation Report\n\n#### Tar...,NaN
...,...,...,...,...,...,...
115,1562,CEM,necessity_robustness_selection,\nYou are an AI assistant that analyzes strugg...,### Report on Student Performance Analysis\n\n...,NaN
116,1562,CEM,contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,**Report on Student Performance Prediction Ana...,NaN
117,1562,CEM,base_contrastive_explanation,\nYou are an AI assistant that analyzes strugg...,### Contrastive Explanation Analysis Report\n\...,NaN
118,1562,CEM,statistical_relevance,\nYou are an AI assistant that analyzes strugg...,**Report on Student Performance Analysis**\n\n...,NaN
